### Analyzing soccer players

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Analyzing soccer players") \
    .getOrCreate()

In [2]:
players = spark.read\
               .format("csv")\
               .option("header", "true")\
               .load("../datasets/player.csv")

In [3]:
players.printSchema()

root
 |-- id: string (nullable = true)
 |-- player_api_id: string (nullable = true)
 |-- player_name: string (nullable = true)
 |-- player_fifa_api_id: string (nullable = true)
 |-- birthday: string (nullable = true)
 |-- height: string (nullable = true)
 |-- weight: string (nullable = true)



In [5]:
player_attributes = spark.read\
                         .format("csv")\
                         .option("header", "true")\
                         .load("../datasets/Player_Attributes.csv")

#### Player attributes

* Have values across multiple years
* Can be associated with a particular player using the **player_api_id** column
* Different attributes are valuable for different kinds of players i.e strikers, midfields, goalkeepers

In [7]:
players.count() , player_attributes.count()

(11060, 183978)

In [8]:
player_attributes.select('player_api_id')\
                 .distinct()\
                 .count()

11060

### Cleaning Data

In [9]:
players = players.drop('id', 'player_fifa_api_id')
players.columns

['player_api_id', 'player_name', 'birthday', 'height', 'weight']

According to our requirement there are certain traits which we are not at all going to use in this entire program<br>
So its better to remove those traits to make our dataset less bulky

In [10]:
player_attributes = player_attributes.drop(
    'id', 
    'player_fifa_api_id', 
    'preferred_foot',
    'attacking_work_rate',
    'defensive_work_rate',
    'crossing',
    'jumping',
    'sprint_speed',
    'balance',
    'aggression',
    'short_passing',
    'potential'
)
player_attributes.columns

['player_api_id',
 'date',
 'overall_rating',
 'finishing',
 'heading_accuracy',
 'volleys',
 'dribbling',
 'curve',
 'free_kick_accuracy',
 'long_passing',
 'ball_control',
 'acceleration',
 'agility',
 'reactions',
 'shot_power',
 'stamina',
 'strength',
 'long_shots',
 'interceptions',
 'positioning',
 'vision',
 'penalties',
 'marking',
 'standing_tackle',
 'sliding_tackle',
 'gk_diving',
 'gk_handling',
 'gk_kicking',
 'gk_positioning',
 'gk_reflexes']

In [11]:
player_attributes = player_attributes.dropna()
players = players.dropna()

In [12]:
players.count() , player_attributes.count()

(11060, 181265)

#### Extract year information into a separate column

In [13]:
from pyspark.sql.functions import udf

In [14]:
year_extract_udf = udf(lambda date: date.split('-')[0])

player_attributes = player_attributes.withColumn(
    "year",
    year_extract_udf(player_attributes.date)
)

In [15]:
player_attributes = player_attributes.drop('date')

In [16]:
player_attributes.columns

['player_api_id',
 'overall_rating',
 'finishing',
 'heading_accuracy',
 'volleys',
 'dribbling',
 'curve',
 'free_kick_accuracy',
 'long_passing',
 'ball_control',
 'acceleration',
 'agility',
 'reactions',
 'shot_power',
 'stamina',
 'strength',
 'long_shots',
 'interceptions',
 'positioning',
 'vision',
 'penalties',
 'marking',
 'standing_tackle',
 'sliding_tackle',
 'gk_diving',
 'gk_handling',
 'gk_kicking',
 'gk_positioning',
 'gk_reflexes',
 'year']

#### Filter to get all players who were active in the year 2016

In [17]:
pa_2016 = player_attributes.filter(player_attributes.year == 2016)

In [18]:
pa_2016.count()

14098

In [19]:
pa_2016.select(pa_2016.player_api_id)\
       .distinct()\
       .count()

5586

#### Find the best striker in the year 2016

* Consider the scores for finishing, shot_power and acceleration to determine this
* There can be more than one entry for a player in the year (multiple seasons, some teams make entries per quarter)
* Find the average scores across the multiple records

In [20]:
pa_striker_2016 = pa_2016.groupBy('player_api_id')\
                       .agg({
                           'finishing':"avg",
                           "shot_power":"avg",
                           "acceleration":"avg"
                       })

In [21]:
pa_striker_2016.count()

5586

In [22]:
pa_striker_2016.show(5)

+-------------+-----------------+-----------------+---------------+
|player_api_id|   avg(finishing)|avg(acceleration)|avg(shot_power)|
+-------------+-----------------+-----------------+---------------+
|       309726|75.44444444444444|74.11111111111111|           76.0|
|        26112|             53.0|             51.0|           76.0|
|        38433|            68.25|             74.0|           74.0|
|       295060|             25.0|             62.0|           40.0|
|       161396|             29.0|             72.0|           69.0|
+-------------+-----------------+-----------------+---------------+
only showing top 5 rows



In [23]:
pa_striker_2016 = pa_striker_2016.withColumnRenamed("avg(finishing)","finishing")\
                                 .withColumnRenamed("avg(shot_power)","shot_power")\
                                 .withColumnRenamed("avg(acceleration)","acceleration")

#### Find an aggregate score to represent how good a particular player is

* Each attribute has a weighing factor
* Find a total score for each striker

In [24]:
weight_finishing = 1
weight_shot_power = 2
weight_acceleration = 1

total_weight = weight_finishing + weight_shot_power + weight_acceleration

In [27]:
strikers = pa_striker_2016.withColumn("striker_grade",
                                      (pa_striker_2016.finishing * weight_finishing + \
                                       pa_striker_2016.shot_power * weight_shot_power+ \
                                       pa_striker_2016.acceleration * weight_acceleration) / total_weight)

In [28]:
strikers = strikers.drop('finishing',
                         'acceleration',
                         'shot_power'
)

In [31]:
strikers = strikers.filter(strikers.striker_grade > 70)\
                   .sort(strikers.striker_grade.desc())
    
strikers.show(10)

+-------------+-----------------+
|player_api_id|    striker_grade|
+-------------+-----------------+
|        20276|            89.25|
|        37412|             89.0|
|        38817|            88.75|
|        32118|            88.25|
|        31921|             87.0|
|        30834|            86.75|
|       303824|85.10714285714286|
|       129944|             85.0|
|       150565|            84.75|
|       158263|            84.75|
+-------------+-----------------+
only showing top 10 rows



#### Find name and other details of the best strikers

* The information is present in the *players* dataframe
* Will involve a join operation between *players* and *strikers*

In [33]:
strikers.count(), players.count()

(1609, 11060)

#### Joining dataframes

In [35]:
striker_details = players.join(strikers, players.player_api_id == strikers.player_api_id)

In [36]:
striker_details.columns

['player_api_id',
 'player_name',
 'birthday',
 'height',
 'weight',
 'player_api_id',
 'striker_grade']

In [37]:
striker_details.count()

1609

In [38]:
striker_details = players.join(strikers, ['player_api_id'])

In [39]:
striker_details.show(5)

+-------------+--------------+-------------------+------+------+-------------+
|player_api_id|   player_name|           birthday|height|weight|striker_grade|
+-------------+--------------+-------------------+------+------+-------------+
|        20276|          Hulk|1986-07-25 00:00:00|180.34|   187|        89.25|
|        37412| Sergio Aguero|1988-06-02 00:00:00|172.72|   163|         89.0|
|        38817|  Carlos Tevez|1984-02-05 00:00:00|172.72|   157|        88.75|
|        32118|Lukas Podolski|1985-06-04 00:00:00|182.88|   183|        88.25|
|        31921|   Gareth Bale|1989-07-16 00:00:00|182.88|   163|         87.0|
+-------------+--------------+-------------------+------+------+-------------+
only showing top 5 rows

